In [1]:
import pandas as pd
import numpy as np
import re as re
from collections import OrderedDict
import os

#The following code take a list of emails (from a CSV) and first apply diferents data tranformation (lower, fillna, strip, etc.) in order to return a clean data frame. After validate that the elements contains '@', if they have not, removes it.
At the second part of the proces validat the email's quality. For that cut the diferents parts of the email (user, domain, extention) and work on each of these in order to create a score for the final analysis.

Considerations:

a) 

In [2]:
# Lectura del CSV y guardado en una variable
def READ_CSV(x):
    return pd.read_csv(x, sep=";", encoding='latin-1')

FILE = READ_CSV('listado_emails.csv')

# Guardo los emails en una sola colimna de un DF
FILE_DF = pd.DataFrame()
FILE_DF[0] = FILE

# Paso la columna con los email del DF a una lista
email_list = []
email_list = FILE_DF[0].values.tolist()

   

In [3]:
print(email_list)


['nicolas-rigueiro@hoymail.com', 'mariano18@gmail.com', 'luis@noexite.com', '21321421', 'sadadadad', '.-.-.-.', 'juan,ignacio@gmail.com', '123132131@1231413.com', 'franciso@inventado.com.ar.ar', 'notiene@noleinteresa.com.ar']


In [4]:
email_list = [x.lower() for x in email_list]

print(email_list)

['nicolas-rigueiro@hoymail.com', 'mariano18@gmail.com', 'luis@noexite.com', '21321421', 'sadadadad', '.-.-.-.', 'juan,ignacio@gmail.com', '123132131@1231413.com', 'franciso@inventado.com.ar.ar', 'notiene@noleinteresa.com.ar']


# Transformation:

In [5]:
def list_transformation(x):
   """_summary_

   Args:
       x (_type_): _description_

   Returns:
       _type_: _description_
   """


   try:
      new_email_list = [email for email in x if email is not None]        # Remove the null values

      new_email_list  = [email.lower() for email in new_email_list ]                         # All to lower
           
      new_email_list  = [x if "@" in x else '' for x in new_email_list]   # Identificate the elementes that contains a '@' and remove if have not it                                
    
      new_list_email_1 = []                                            # Remove special caracters to the end of the emails
      special_caracters = ['.', '-', ',','"','/',';',':','_']
      for email in new_email_list :
         if email[-1:] in special_caracters:
           new_list_email_1.append(email[:-1])
         else:
           new_list_email_1.append(email)

      return new_list_email_1
   
   except: print('The file has more than 1 column')
       


In [6]:
list_transformation(email_list)

['nicolas-rigueiro@hoymail.com',
 'mariano18@gmail.com',
 'luis@noexite.com',
 '',
 '',
 '',
 'juan,ignacio@gmail.com',
 '123132131@1231413.com',
 'franciso@inventado.com.ar.ar',
 'notiene@noleinteresa.com.ar']

# Structure validation:

In [7]:
# Split usre - domains to validated:
list_email_4 = pd.DataFrame()
list_email_4[0] = list_transformation(email_list)
list_email_5 = list_email_4[0].str.split('@', n=2, expand=True)

# Domains validate:
list_extention = ['hotmail.com','hotmail.com.ar','live.com','outlook.com','yahoo.com','yahoo.com.ar','gmail.com']

quality_domain = []
for x in list_email_5[1]: 
    if x in list_extention:
        quality_domain.append(0)
    elif x == (None):
        quality_domain.append('')
    elif x not in list_extention:
        quality_domain.append(1)
           
list_email_5['quality_domain'] = quality_domain
list_email_5[1].fillna('', inplace = True)

# Etention validate:
quality_extention = []
for x in list_email_5[1]:
    if '.com' in x:
        quality_extention.append(0)
    elif x == '':
        quality_extention.append('')
    elif '.com' not in x:
        quality_extention.append(1)

list_email_5['quality_extention'] = quality_extention

# Domain doubtfoul (user = domain):
list_email_split = list_email_5[1].str.split('.', n = 5, expand = True)
list_email_5['domain'] = list_email_split[0]
list_email_5['extention_1'] = list_email_split[1]
list_email_5['extention_2'] = list_email_split[2]

list_email_5['extention_1'].fillna('', inplace = True)
list_email_5['extention_2'].fillna('', inplace = True)

quality_domain_doubtful = []  

for x1, x2 in zip(list_email_5[0], list_email_5['domain']):
    if x1 == (''):
       quality_domain_doubtful.append('') 
    elif x1 != x2:
       quality_domain_doubtful.append(0)
    elif x1 == x2:
       quality_domain_doubtful.append(1)

list_email_5['quality_domain_doubtful'] = quality_domain_doubtful
    
# User doubtful (user == 'no tiene'):
list_extention_users = ['notiene','no_tiene','no.tiene','nousa','no_usa','no.usa','noexiste','no_existe','no.existe']

quality_user_doubtful = []
for x in list_email_5[0]: 
    if x in list_extention_users:
        quality_user_doubtful.append(2)
    elif x == (''):
        quality_user_doubtful.append('')
    elif x not in list_extention_users:
        quality_user_doubtful.append(0)
           
list_email_5['quality_user_doubtful'] = quality_user_doubtful

# Final quality analysis:
final = []         
final = list_email_5['quality_domain'] + list_email_5['quality_extention'] + list_email_5['quality_domain_doubtful'] + list_email_5['quality_user_doubtful']

final_quality = []
for x in final:
    if x == 0:
        final_quality.append('100%')
    elif x == 1:
        final_quality.append('70%')
    elif x == 2:
        final_quality.append('40%')
    elif x == 3:
        final_quality.append('30%')
    elif x == 4:
        final_quality.append('20%')
    elif x == 5:
        final_quality.append('0%')            
    elif x == (''):
        final_quality.append('')        

list_email_5['final_quality'] = final_quality

# Orden del Output:
list_email_5 = list_email_5.rename(columns = {0:'user',1:'full_domain'})
list_email_5 = list_email_5[['user','full_domain','domain','extention_1','extention_2','quality_domain','quality_extention','quality_domain_doubtful','quality_user_doubtful','final_quality']]


In [10]:
print(final)
print(final_quality)

0    1
1    0
2    1
3     
4     
5     
6    0
7    1
8    1
9    3
dtype: object
['70%', '100%', '70%', '', '', '', '100%', '70%', '70%', '30%']


In [9]:
list_email_5.head(30)

,user,full_domain,domain,extention_1,extention_2,quality_domain,quality_extention,quality_domain_doubtful,quality_user_doubtful,final_quality
0,nicolas-rigueiro,hoymail.com,hoymail,com,,1,0,0,0,70%
1,mariano18,gmail.com,gmail,com,,0,0,0,0,100%
2,luis,noexite.com,noexite,com,,1,0,0,0,70%
3,,,,,,,,,,
4,,,,,,,,,,
5,,,,,,,,,,
6,"juan,ignacio",gmail.com,gmail,com,,0,0,0,0,100%
7,123132131,1231413.com,1231413,com,,1,0,0,0,70%
8,franciso,inventado.com.ar.ar,inventado,com,ar,1,0,0,0,70%
9,notiene,noleinteresa.com.ar,noleinteresa,com,ar,1,0,0,2,30%
